Set Colab to GPU Mode if you are training the models!

### Mounting Google Drive

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# cd 'drive/MyDrive/IIT J Summer Internship 2022/Code/Neural-Uncertainty-Representation/'

### Importing Necessary Libraries


In [46]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow.keras.layers import Dense, Dropout, LSTM, TimeDistributed
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
import os

### Importing Data
We import the data of a old subject's 3D-Tensor Data of dimensions (TR, Voxel, Number of Subjects) for the 5 different ROIs as follows:
- **Prefrontal areas:** dlPFC, vlPFC, lOFC, dmPFC, mPFC, mOFC
- **Default-mode areas:** mPFC, ACC, PCC, Precuneus, mOFC
- **Sensory areas:** VC
- **Multisensory area:** pSTS
- **Some other areas:** TP, IPL, mCC, Put, PCG, Nac, INS





In [47]:
# file_names = [name for name in os.listdir('/content/drive/MyDrive/IIT J Summer Internship 2022/Hitchcock/Data/YOUNG/Voxel_BOLD/Numpy') if name.endswith("npy") and ("(1)" not in name)]
file_names = [name for name in os.listdir('./Data/OLD/Voxel_BOLD/Numpy') if name.endswith("npy") and ("(1)" not in name)]
file_names = [name for name in file_names if ("537" not in name)]

In [48]:
ROI_names = [file_name.replace("data_","").replace(".npy","") for file_name in file_names]

In [49]:
data_ori = list()
for file_name in file_names:
    # data_ori.append(np.load('/content/drive/MyDrive/IIT J Summer Internship 2022/Hitchcock/Data/YOUNG/Voxel_BOLD/Numpy/'+file_name))
    data_ori.append(np.load('./Data/OLD/Voxel_BOLD/Numpy/'+file_name))

### Data Preparation

Now, here we prepare the data to be fed to the Neural Network Architechtures. Following on the previous developement of the LSTM model to predict BOLD, as a proxy to calculate **Uncertainty**, I use entropy at each time step.

Right from the starting we decide some of the data preparation parameters, so that we can use them as we require.

In [50]:
TRAIN_BATCH_SIZE = 10
TEST_BATCH_SIZE = 8
LOOK_AHEADS = list(range(1, 2)) 
NUM_TEST_SUBS = 8

We will start a basic data preparation where we will average across all the voxels for a ROI and fit the model which predicts this average BOLD value. Later we will avoid doing an average since, all voxels in a ROI is not equally important and hence we will want to predict the BOLD values of several voxels simultaneously.

In [51]:
data = [np.mean(dat, axis=1, keepdims=True) for dat in data_ori]

Let's Start the Data Preparation for lOFC later we will iterate the same procedure for the other ROIs.

In [52]:
print(ROI_names[-1], "Average BOLD Shape:", data[-1].shape)

dlPFC Average BOLD Shape: (189, 1, 98)


Below, we see the average BOLD time series plot for all the ROIs.

We have changed the data in the form (Number of Subjects, TR, Voxels) below.

In [53]:
data = [np.transpose(dat, (2,0,1)) for dat in data]

In [54]:
print(ROI_names[-1], "Average BOLD Shape:", data[-1].shape)

dlPFC Average BOLD Shape: (98, 189, 1)


In [55]:
# for i in range(len(data)):  
#   plt.figure(figsize=(10, 8))
#   plt.plot(data[i][0,:,0])
#   plt.title("Average BOLD of 1st Subject for "+ROI_names[i])
#   plt.xlabel("TRs")
#   plt.ylabel("Average BOLD(over voxels)")
#   plt.savefig("./Plots/YOUNG/AVG-BOLD/AVG-BOLD-"+ROI_names[i]+".png")

We split the dataset of $98$ young subjects into $90$ subjects in the training data and $8$ subject in the test data.

In [56]:
train_data_unscaled = [dat[:-NUM_TEST_SUBS] for dat in data]
test_data_unscaled = [dat[-NUM_TEST_SUBS:] for dat in data]  

We scaled the BOLD values of each voxel in the train data between 0 and 1 for each voxel. We use the same MinMixScaler fit on the train data on the test data.

In [57]:
scalers = list()
for i in range(len(ROI_names)):
  scalers.append(MinMaxScaler(feature_range = (0, 1)))
train_data = [sc.fit_transform(dat[:,:,0]) for sc, dat in zip(scalers, train_data_unscaled)] 
test_data = [sc.transform(dat[:,:,0]) for sc, dat in zip(scalers, test_data_unscaled)]

In [58]:
train_data = [np.expand_dims(dat, axis = -1) for dat in train_data]
test_data = [np.expand_dims(dat, axis = -1) for dat in test_data]

In [59]:
print(ROI_names[-1], "Train Data Shape: ", train_data[-1].shape)
print(ROI_names[-1], "Test Data Shape: ", test_data[-1].shape)

dlPFC Train Data Shape:  (90, 189, 1)
dlPFC Test Data Shape:  (8, 189, 1)


Now, for the purpose of binning which we will use for calculating entropy, each time point in BOLD signal, after standard scaling needs to be split up into regions based on the value being some standard deviation away from $0$. So, how to split the data is what we will decide here. I go with the split such that each bin contains equal number of values.

In [60]:
NUM_BINS = 30 # Per ROI how many bins should be there

In [61]:
def equalObs(x, nbin):
  nlen = len(x)
  return np.interp(np.linspace(0, nlen, nbin + 1), np.arange(nlen), np.sort(x))

In [62]:
roi_bins = list()
train_target = list()
for roi_data in train_data:
  _, bins, _ = plt.hist(np.squeeze(np.concatenate(roi_data)), equalObs(np.squeeze(np.concatenate(roi_data)), NUM_BINS))
  # _, bins, _ = plt.hist(np.squeeze(np.concatenate(roi_data)), NUM_BINS)
  plt.close()
  roi_bins.append(bins)
  train_target.append(np.sum(roi_data > np.tile(bins, roi_data.shape), axis=-1, keepdims=True))

In [63]:
len(roi_bins[0])

31

Below, we generate the binned plots to see how the results look like for the first subject.

In [64]:
# for i in range(len(data)):  
#   plt.figure(figsize=(10, 8))
#   plt.plot(train_data[i][0,:,0], label='Actual')
#   plt.plot(roi_bins[i][train_target[i][0,:,0]], label='Binned')
#   plt.legend()
#   plt.title("Average BOLD of 1st Subject for "+ROI_names[i]+" with "+str(NUM_BINS)+" binned targets (Train Data Based Bins)")
#   plt.xlabel("TRs")
#   plt.ylabel("Average BOLD(over voxels)[Scaled between 0 and 1]")
#   plt.savefig("./Plots/OLD/AVG-BOLD/AVG-BOLD-SCALED-TRAIN-"+str(NUM_BINS)+"-BINS-"+ROI_names[i]+".png")

Now, from the bins generated from the training data, I will use them to bin the test data

In [65]:
test_target = list()
for roi_data, bins in zip(test_data, roi_bins):
  test_target.append(np.sum(roi_data > np.tile(bins, roi_data.shape), axis=-1, keepdims=True))

We now build a data generator which would produce mini-batches during the training of the neural network.

In [66]:
class KerasBatchGenerator(object):
    def __init__(self, data, bins_data, num_TRs, batch_size, look_ahead=1):
        self.data = data
        self.bins_data = bins_data
        self.num_TRs = num_TRs
        self.batch_size = batch_size
        self.current_idx = 0
        self.look_ahead = look_ahead

    def generate(self):
        while True:
            if self.current_idx + self.batch_size >= self.data.shape[0]:
              self.current_idx = 0
            x = self.data[self.current_idx:(self.current_idx+self.batch_size),:(-self.look_ahead),:]
            y = self.bins_data[self.current_idx:(self.current_idx+self.batch_size),self.look_ahead:,:]
            self.current_idx = self.current_idx + self.batch_size
            yield x, y 

In [67]:
train_data_generator, test_data_generator = list(), list()
for LOOK_AHEAD in LOOK_AHEADS:
    train_data_generator.append([KerasBatchGenerator(dat, bin_dat, dat.shape[1], TRAIN_BATCH_SIZE, LOOK_AHEAD) for dat, bin_dat in zip(train_data, train_target)])
    test_data_generator.append([KerasBatchGenerator(dat, bin_dat, dat.shape[1], TEST_BATCH_SIZE, LOOK_AHEAD) for dat, bin_dat in zip(test_data, test_target)])

In [68]:
batch_xs, batch_ys = list(), list()
for LOOK_AHEAD in LOOK_AHEADS:
    batch = next(train_data_generator[LOOK_AHEAD-1][-1].generate())
    batch_xs.append(batch[0])
    batch_ys.append(batch[1])
    print(f"{ROI_names[-1]} Train Batch X Shape for {LOOK_AHEAD} Ahead: ", batch_xs[LOOK_AHEAD-1].shape)
    print(f"{ROI_names[-1]} Train Batch y Shape for {LOOK_AHEAD} Ahead: ", batch_ys[LOOK_AHEAD-1].shape)

dlPFC Train Batch X Shape for 1 Ahead:  (10, 188, 1)
dlPFC Train Batch y Shape for 1 Ahead:  (10, 188, 1)


In [69]:
batch_xs, batch_ys = list(), list()
for LOOK_AHEAD in LOOK_AHEADS:
    batch = next(test_data_generator[LOOK_AHEAD-1][-1].generate())
    batch_xs.append(batch[0])
    batch_ys.append(batch[1])
    print(f"{ROI_names[-1]} Test Batch X Shape for {LOOK_AHEAD} Ahead: ", batch_xs[LOOK_AHEAD-1].shape)
    print(f"{ROI_names[-1]} Test Batch y Shape for {LOOK_AHEAD} Ahead: ", batch_ys[LOOK_AHEAD-1].shape)

dlPFC Test Batch X Shape for 1 Ahead:  (8, 188, 1)
dlPFC Test Batch y Shape for 1 Ahead:  (8, 188, 1)


### Building Model

We will start off by building a LSTM Model:
- `Number of LSTM Layers = 1`
- `Number of Dense Layers = 2`
- `hidden units = 32`
- `dropout = 0.3`

In [27]:
HIDDEN_UNITS = 64
DROPOUT_PROB = 0.3
LSTM_LAYERS = 1
DENSE_LAYERS = 3

In [28]:
def avg_bold_model():
  model = Sequential()
  for _ in range(LSTM_LAYERS):
    model.add(LSTM(HIDDEN_UNITS, return_sequences=True))
  model.add(Dropout(0.3))
  for _ in range(DENSE_LAYERS-1):
    model.add(TimeDistributed(Dense(HIDDEN_UNITS, activation='relu')))
  model.add(TimeDistributed(Dense(NUM_BINS + 1, activation='softmax')))
  return model

In [29]:
for LOOK_AHEAD in LOOK_AHEADS:
    print(f"***{LOOK_AHEAD} Look Ahead Model***")
    sample_model = avg_bold_model()
    sample_model.build(input_shape=batch_xs[LOOK_AHEAD-1].shape)
    sample_model.summary()

***1 Look Ahead Model***
Metal device set to: Apple M1 Max


2022-11-10 22:53:40.714420: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-10 22:53:40.714526: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (8, 188, 64)              16896     
                                                                 
 dropout (Dropout)           (8, 188, 64)              0         
                                                                 
 time_distributed (TimeDistr  (8, 188, 64)             4160      
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (8, 188, 64)             4160      
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (8, 188, 31)             2015      
 tributed)                                                       
                                                        

In [30]:
models = [[avg_bold_model() for _ in ROI_names] for _ in LOOK_AHEADS]

#### Training Model

If you haven't already trained the models then uncomment and train your models which will be saved and will be loaded for inference later.

In [31]:
for i in range(len(LOOK_AHEADS)):
    for j in range(len(ROI_names)):
        models[i][j].compile(optimizer = 'adam', loss = tf.keras.losses.SparseCategoricalCrossentropy())

In [32]:
EPOCHS = 300
checkpoint_filepaths = [[f"./Models/OLD/LSTM/AVG-BINNED-BOLD/{name}/model_weights-best-val-loss-{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-{LOOK_AHEAD}Ahead-{name}" for name in ROI_names] for LOOK_AHEAD in LOOK_AHEADS]
model_checkpoint_callbacks = [[tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepat,
    save_weights_only=True,
    monitor = "val_loss",
    mode='min',
    save_best_only=True) for checkpoint_filepat in checkpoint_filepath] for checkpoint_filepath in checkpoint_filepaths]

In [33]:
def plot_train_test_loss(history, model_name, i):  
  plt.figure(figsize=(10,10))
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title(model_name+' loss')
  plt.ylabel('loss')
  plt.xlabel('epochs')
  plt.legend(['train', 'val'], loc='upper left')
  plt.savefig("./Plots/OLD/LSTM/AVG-BINNED-BOLD/"+ROI_names[i]+"/"+model_name+"-loss.png")

In [ ]:
histories = list()
with tf.device('/device:GPU:0'):
    for LOOK_AHEAD in LOOK_AHEADS:
        history = list()
        for j in range(len(ROI_names)):
            print(f'***{ROI_names[j]}***')
            hist = models[LOOK_AHEAD-1][j].fit(
                        train_data_generator[LOOK_AHEAD-1][j].generate(),
                        steps_per_epoch=train_data[i].shape[0]//TRAIN_BATCH_SIZE,
                        epochs=EPOCHS,
                        validation_data=test_data_generator[LOOK_AHEAD-1][j].generate(),
                        validation_steps=test_data[i].shape[0]//TEST_BATCH_SIZE,
                        callbacks=[model_checkpoint_callbacks[LOOK_AHEAD-1][j]]
                        )
            models[LOOK_AHEAD-1][j].save_weights(f"./Models/OLD/LSTM/AVG-BINNED-BOLD/{ROI_names[j]}/model_weights-{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-{LOOK_AHEAD}Ahead-{ROI_names[j]}")
            plot_train_test_loss(hist, f"{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-{LOOK_AHEAD}Ahead-{ROI_names[j]}", j)
            history.append(hist)
        histories.append(history)

In [70]:
for LOOK_AHEAD in LOOK_AHEADS:
    for i in range(len(ROI_names)):
        models[LOOK_AHEAD-1][i].load_weights(f"./Models/OLD/LSTM/AVG-BINNED-BOLD/{ROI_names[i]}/model_weights-{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-{LOOK_AHEAD}Ahead-{ROI_names[i]}")

In [71]:
preds_raw = [[models[LOOK_AHEAD-1][i].predict(next(test_data_generator[LOOK_AHEAD-1][i].generate())[0]) for i in range(len(ROI_names))] for LOOK_AHEAD in LOOK_AHEADS]
preds = [[np.argmax(preds_raw[LOOK_AHEAD-1][i], axis=-1) for i in range(len(ROI_names))] for LOOK_AHEAD in LOOK_AHEADS]

1/1 [==============================] - 0s 32ms/step


In [72]:
a_TRs = np.array([56, 67, 73, 81, 134, 142, 155, 167, 174, 180])
d_TRs = np.array([67, 72, 85, 89, 145, 154, 168, 174, 180, 188])
and_TRs = np.union1d(a_TRs, d_TRs)

def highlight_and(act, pred):
  min_y_list = list()
  max_y_list = list()
  for i in range(5):
    min_y_list.append(np.min(np.concatenate([act[a_TRs[2*i]:d_TRs[2*i+1]], pred[a_TRs[2*i]:d_TRs[2*i+1]]])))
    max_y_list.append(np.max(np.concatenate([act[a_TRs[2*i]:d_TRs[2*i+1]], pred[a_TRs[2*i]:d_TRs[2*i+1]]])))
    plt.fill_betweenx(y=[min_y_list[i], max_y_list[i]], x1=a_TRs[2*i], x2=d_TRs[2*i+1], color='gray', alpha=0.1)
    plt.fill_betweenx(y=[min_y_list[i], max_y_list[i]], x1=a_TRs[2*i], x2=a_TRs[2*i+1], color='violet', alpha=0.4)
    plt.fill_betweenx(y=[min_y_list[i], max_y_list[i]], x1=d_TRs[2*i], x2=d_TRs[2*i+1], color='greenyellow', alpha=0.4)

In [73]:
preds[0][-4].shape

(8, 188)

In [74]:
np.squeeze(test_target[0])[:,1:].shape

(8, 188)

In [ ]:
for LOOK_AHEAD in LOOK_AHEADS:
    for i in range(len(ROI_names)):
      
        actual_test_data_output = np.squeeze(test_target[i])[:,LOOK_AHEAD:] 
        pred_test_data_output = preds[LOOK_AHEAD-1][i]
        plt.figure(figsize=(24, 18))

        for j in range(NUM_TEST_SUBS):
            ax = plt.subplot(3, 3, j + 1)
            plt.plot(actual_test_data_output[j,:])
            plt.plot(pred_test_data_output[j,:])
            highlight_and(actual_test_data_output[j,:], pred_test_data_output[j,:])
            plt.xticks(np.concatenate([np.array([0]), and_TRs - LOOK_AHEAD]), np.concatenate([np.array([LOOK_AHEAD]), and_TRs]))
            plt.ylabel("Average Binned BOLD")
            plt.xlabel("TRs")
            plt.title(f"Avg Binned BOLD Preds of {LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-{NUM_BINS}Bin-{LOOK_AHEAD}Ahead for {j+1}th Test Sub's {ROI_names[i]}")
            plt.legend(['actual', 'pred'])
        plt.savefig(f"./Plots/OLD/LSTM/AVG-BINNED-BOLD/{ROI_names[i]}/{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-{NUM_BINS}Bin-{LOOK_AHEAD}Ahead-Preds.png")

In [77]:
TR_ROI_corr_subs, AVG_TR_ROI_corr_subs = list(), list()
for LOOK_AHEAD in LOOK_AHEADS:
    print(f"***Each Test Subject's Correlation Coefficient between {LOOK_AHEAD} ahead predicted and actual average binned BOLD values for ROIs***")
    print()
    avg_r = list()
    ROI_corr_subs = list()
    for i in range(len(ROI_names)):
        actual_test_data_output = np.squeeze(test_target[i])[:,LOOK_AHEAD:] 
        pred_test_data_output = preds[LOOK_AHEAD-1][i]
        corr_subs = list()
        for j in range(NUM_TEST_SUBS):
            corr_subs.append(np.round(np.corrcoef(actual_test_data_output[j,:], pred_test_data_output[j,:])[0,1], 2))
            print(f"{ROI_names[i]} for {j+1}th sub: {corr_subs[j]}")
        ROI_corr_subs.append(corr_subs)
    TR_ROI_corr_subs.append(ROI_corr_subs)
    print()

    print(f"***Average across test subjects Correlation Coefficient between {LOOK_AHEAD} ahead predicted and actual average binned BOLD values for ROIs***")
    print()
    ROI_corr_subs = list()
    for i in range(len(ROI_names)):
        actual_test_data_output = np.squeeze(test_target[i])[:,LOOK_AHEAD:] 
        pred_test_data_output = preds[LOOK_AHEAD-1][i]
        sum_r = 0
        for j in range(NUM_TEST_SUBS):
            correl = np.corrcoef(actual_test_data_output[j,:], pred_test_data_output[j,:])[0,1]
            if not np.isnan(correl):
              sum_r += correl
        ROI_corr_subs.append(np.round(sum_r / NUM_TEST_SUBS, 2))
        print(f"{ROI_names[i]}: {ROI_corr_subs[i]}")
    AVG_TR_ROI_corr_subs.append(ROI_corr_subs)

***Each Test Subject's Correlation Coefficient between 1 ahead predicted and actual average binned BOLD values for ROIs***

INS for 1th sub: 0.03
INS for 2th sub: -0.23
INS for 3th sub: -0.01
INS for 4th sub: -0.17
INS for 5th sub: 0.33
INS for 6th sub: 0.81
INS for 7th sub: 0.39
INS for 8th sub: 0.34
PCC for 1th sub: 0.2
PCC for 2th sub: 0.11
PCC for 3th sub: 0.18
PCC for 4th sub: 0.02
PCC for 5th sub: 0.29
PCC for 6th sub: 0.4
PCC for 7th sub: 0.04
PCC for 8th sub: 0.13
Cau for 1th sub: 0.33
Cau for 2th sub: 0.23
Cau for 3th sub: 0.23
Cau for 4th sub: 0.06
Cau for 5th sub: 0.01
Cau for 6th sub: 0.15
Cau for 7th sub: 0.06
Cau for 8th sub: 0.18
mCC for 1th sub: 0.33
mCC for 2th sub: -0.05
mCC for 3th sub: -0.02
mCC for 4th sub: 0.27
mCC for 5th sub: 0.19
mCC for 6th sub: 0.47
mCC for 7th sub: -0.03
mCC for 8th sub: 0.03
mOFC for 1th sub: 0.75
mOFC for 2th sub: 0.72
mOFC for 3th sub: 0.67
mOFC for 4th sub: 0.56
mOFC for 5th sub: 0.55
mOFC for 6th sub: 0.5
mOFC for 7th sub: 0.78
mOFC for

/Users/rishideychowdhury/Desktop/Neural-Uncertainty-Representation/env/lib/python3.8/site-packages/numpy/lib/function_base.py:2829: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/rishideychowdhury/Desktop/Neural-Uncertainty-Representation/env/lib/python3.8/site-packages/numpy/lib/function_base.py:2830: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [78]:
TR_ROI_corr_subs = np.transpose(np.array(TR_ROI_corr_subs), (1,0,2))
AVG_TR_ROI_corr_subs = np.array(AVG_TR_ROI_corr_subs).T
print(TR_ROI_corr_subs.shape, AVG_TR_ROI_corr_subs.shape)

(19, 1, 8) (19, 1)


In [79]:
# for i in range(len(ROI_names)):
#     plt.figure(figsize=(24, 18))

#     for j in range(NUM_TEST_SUBS-1):

#         ax = plt.subplot(3, 3, j + 1)
#         plt.plot(TR_ROI_corr_subs[i, :, j])
#         plt.xticks(np.arange(1, 16))
#         plt.ylabel("Correlation")
#         plt.xlabel("Ahead TRs")
#         plt.title(f"Avg BOLD Corr of {LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop for {j+1}th Test Sub's {ROI_names[i]}")
#     plt.savefig(f"./Plots/YOUNG/LSTM/AVG-BOLD-TEST-ENLARGED/{ROI_names[i]}/{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-Upto-15Ahead-Corrs.png")

In [80]:
# for i in range(len(ROI_names)):
#     plt.figure(figsize=(10, 10))
#     plt.plot(AVG_TR_ROI_corr_subs[i])
#     plt.xticks(np.arange(1, 16))
#     plt.ylabel("Correlation")
#     plt.xlabel("Ahead TRs")
#     plt.title(f"Avg BOLD Corr of {LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop Across Test Subs")
#     plt.savefig(f"./Plots/YOUNG/LSTM/AVG-BOLD-TEST-ENLARGED/{ROI_names[i]}/{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-Upto-15Ahead-Corrs-Avg-Across-Subs.png")

In [81]:
# for LOOK_AHEAD in LOOK_AHEADS:
#     print(f"***Model's Final MSE(Train) Loss between {LOOK_AHEAD} ahead predicted and actual average BOLD values for ROIs for the Train Subjects***")
#     print()
#     for i in range(len(ROI_names)):
#         print(ROI_names[i]+":", np.round(histories[LOOK_AHEAD-1][i].history['loss'][-1], 5))
#     print()
#     print(f"***Model's Final MSE(Validation) Loss between {LOOK_AHEAD} ahead predicted and actual average BOLD values for ROIs for the Test Subjects***")
#     print()
#     for i in range(len(ROI_names)):
#         print(ROI_names[i]+":", np.round(histories[LOOK_AHEAD-1][i].history['val_loss'][-1], 5))

In [82]:
CXT_subs, AVG_CXT_subs = list(), list()
for LOOK_AHEAD in LOOK_AHEADS:
    print(f"***Model's Final CXT(Validation) Loss between {LOOK_AHEAD} ahead predicted and actual avg voxel binned BOLD values for ROIs for the Test Subjects***")
    print()
    ROI_CXT_subs, ROI_AVG_CXT_subs = list(), list()
    for i in range(len(ROI_names)):
        actual_test_data_output = np.squeeze(test_target[i])[:,LOOK_AHEAD:] 
        pred_test_data_output = preds_raw[LOOK_AHEAD-1][i]
        ROI_CXT_subs.append(np.mean(tf.keras.losses.sparse_categorical_crossentropy(actual_test_data_output, pred_test_data_output).numpy(), axis=-1))
        print(ROI_names[i]+" for Each Validation Subject:", ROI_CXT_subs[i])
        ROI_AVG_CXT_subs.append(np.mean(ROI_CXT_subs[i]))
        print(ROI_names[i]+" :", ROI_AVG_CXT_subs[i])
    CXT_subs.append(ROI_CXT_subs)
    AVG_CXT_subs.append(ROI_AVG_CXT_subs)

***Model's Final CXT(Validation) Loss between 1 ahead predicted and actual avg voxel binned BOLD values for ROIs for the Test Subjects***

INS for Each Validation Subject: [1.2920655 1.0232675 1.6760383 1.0652968 1.1702557 1.4276941 1.568854
 1.4180449]
INS : 1.3301895
PCC for Each Validation Subject: [1.0145931 1.4911249 1.7006091 0.6393978 1.6717345 1.4762906 1.2830639
 1.4606388]
PCC : 1.3421816
Cau for Each Validation Subject: [1.7296821  1.7306074  1.4725024  0.83410937 1.459614   1.5126748
 0.84698    0.8124861 ]
Cau : 1.299832
mCC for Each Validation Subject: [1.3964611  0.9572339  0.8432738  1.0977805  1.7070559  1.0784343
 1.5812386  0.90127426]
mCC : 1.1953441
mOFC for Each Validation Subject: [1.6976    1.6000735 1.6157185 1.7271446 1.7062753 1.058186  1.5748552
 1.5965158]
mOFC : 1.572046
l_OFC for Each Validation Subject: [1.5005947 1.9165318 1.93931   1.8143177 1.770541  1.5569768 1.6765977
 1.5965339]
l_OFC : 1.7214255
vlPFC for Each Validation Subject: [1.1649641  0.723

In [83]:
CXT_subs = np.transpose(np.array(CXT_subs), (1, 0, 2))
AVG_CXT_subs = np.array(AVG_CXT_subs).T
print(CXT_subs.shape, AVG_CXT_subs.shape)

(19, 1, 8) (19, 1)


In [84]:
# for i in range(len(ROI_names)):
#     plt.figure(figsize=(24, 18))

#     for j in range(NUM_TEST_SUBS-1):

#         ax = plt.subplot(3, 3, j + 1)
#         plt.plot(MSE_subs[i, :, j])
#         plt.xticks(np.arange(1, 16))
#         plt.ylabel("MSE Loss")
#         plt.xlabel("Ahead TRs")
#         plt.title(f"Avg BOLD MSE Losses of {LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop for {j+1}th Test Sub's {ROI_names[i]}")
#     plt.savefig(f"./Plots/YOUNG/LSTM/AVG-BOLD-TEST-ENLARGED/{ROI_names[i]}/{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-Upto-15Ahead-Losses.png")

In [85]:
# for i in range(len(ROI_names)):
#     plt.figure(figsize=(10, 10))
#     plt.plot(AVG_MSE_subs[i])
#     plt.xticks(np.arange(1, 16))
#     plt.ylabel("MSE Loss")
#     plt.xlabel("Ahead TRs")
#     plt.title(f"Avg BOLD MSE Losses of {LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop Across Test Subs")
#     plt.savefig(f"./Plots/YOUNG/LSTM/AVG-BOLD-TEST-ENLARGED/{ROI_names[i]}/{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-Upto-15Ahead-Losses-Avg-Across-Subs.png")

In [ ]:
for LOOK_AHEAD in LOOK_AHEADS:
  for i in range(len(ROI_names)):
    entropy = -1 * np.sum(preds_raw[LOOK_AHEAD-1][i]*np.log(preds_raw[LOOK_AHEAD-1][i]), axis=-1)
    plt.figure(figsize=(24, 18))
    for j in range(NUM_TEST_SUBS):
      
      ax = plt.subplot(3, 3, j + 1)
      plt.plot(entropy[j,:])
      highlight_and(entropy[j,:], entropy[j,:])
      plt.xticks(np.concatenate([np.array([0]), and_TRs - LOOK_AHEAD]), np.concatenate([np.array([LOOK_AHEAD]), and_TRs]))
      plt.ylabel("Entropy of preds avg binned BOLD")
      plt.xlabel("TRs")
      plt.title(f"Entropy Avg BOLD of {LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-{NUM_BINS}Bin-{LOOK_AHEAD}Ahead for {j+1}th Test Sub's {ROI_names[i]}")
    plt.savefig(f"./Plots/OLD/LSTM/AVG-BINNED-BOLD/{ROI_names[i]}/{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-{NUM_BINS}Bin-{LOOK_AHEAD}Ahead-Preds-Entropy.png")

In [ ]:
for LOOK_AHEAD in LOOK_AHEADS:
  entropy = [-1 * np.sum(preds_raw[LOOK_AHEAD-1][i]*np.log(preds_raw[LOOK_AHEAD-1][i]), axis=-1) for i in range(len(ROI_names))]
  entropy_avg = [np.mean(entropy_ind, axis=0) for entropy_ind in entropy]
  for i in range(len(ROI_names)):
    plt.figure(figsize=(10, 10))
    plt.plot(entropy_avg[i])
    highlight_and(entropy_avg[i], entropy_avg[i])
    plt.xticks(np.concatenate([np.array([0]), and_TRs - LOOK_AHEAD]), np.concatenate([np.array([LOOK_AHEAD]), and_TRs]))
    plt.ylabel("Entropy of preds avg binned BOLD")
    plt.xlabel("TRs")
    plt.title(f"Entropy Avg BOLD of {LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-{NUM_BINS}Bin-{LOOK_AHEAD}Ahead for All Test Subs Averaged {ROI_names[i]}")
    plt.savefig(f"./Plots/OLD/LSTM/AVG-BINNED-BOLD/{ROI_names[i]}/{LSTM_LAYERS}L-{HIDDEN_UNITS}H-{DENSE_LAYERS}D-{int(DROPOUT_PROB*10)}Drop-{NUM_BINS}Bin-{LOOK_AHEAD}Ahead-Preds-All-Averaged-Entropy.png")